In [66]:
import pikepdf


In [ ]:

def remove_pdf_password(input_pdf, output_pdf, password):
    try:
        # Open the encrypted PDF with the provided password
        with pikepdf.open(input_pdf, password=password) as pdf:
            # Save the PDF without the password
            pdf.save(output_pdf)

        print(f"Password removed successfully. Saved to {output_pdf}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_pdf = "pnb_statement.pdf"  # Path to the input PDF
output_pdf = f'1{input_pdf}'  # Path to the output PDF
password = "02172021000234"  # Password for the encrypted PDF

remove_pdf_password(input_pdf, output_pdf, password)


In [ ]:
import mlflow

mlflow.login()

In [ ]:
with mlflow.start_run(run_id=run_id):
  # mlflow.log_param("row_count",df_merged.count())
  mlflow.log_dict(null_counts, "null_counts")

In [2]:
from mlflow import MlflowClient

client = MlflowClient()
# exp_id = client.create_experiment("/Users/i-sachin.bahuleyan@d2kindia.com/PD_modelsv6")
# experiment = client.get_experiment('2953816066572483')

In [109]:
run = client.get_run('2a1f6d3066984315af874674efc29f37')


In [ ]:
run.data.metrics

In [115]:
p = "artifacts/roc_curve/roc_curve"+".png"

In [50]:
# cleaned_string = string.replace('"', '').replace("'", "").replace('`', '')

feat = run.data.params['features'].replace('"','').replace("'",'').replace('[','').replace(']','').replace(",",'').split()


In [93]:
import sys
sys.path.insert(0,"..")

from dashboard.components import helper

In [94]:
obj = helper.Helper()

In [ ]:
print(obj.run_id)

In [3]:
art = client.list_artifacts('5db55e7be5504734a98e6b2a45518ecc')

In [ ]:
art

In [ ]:
art[2].path.startswith('feature_importances')

In [ ]:
if art[0].path in ['XGBClassifier', 'LogisticRegression', 'DtClassifier']:
    # Do something
    print('yes')
else:
    print('no')

In [20]:
pat = client.download_artifacts('d8efee6cdc2f40fa81508e5a7c43f367',art[2].path,dst_path='../dashboard/assets')

In [ ]:
pat

In [9]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("PySpark Test") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.memory", "10g") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
spark.conf.set("fs.s3a.access.key", "TYUFUFGRYT44XA5CJJQB")
spark.conf.set("fs.s3a.secret.key", "DB3AKW1PDSBGGXR7FNPMG7XMTT5E5DYZ6NI4H56Y")
spark.conf.set("fs.s3a.endpoint", "https://mum-objectstore.e2enetworks.net")
spark.conf.set("fs.s3a.path.style.access", "true")
spark.conf.set("fs.s3a.connection.ssl.enabled", "true")
# spark.conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.conf.set("spark.kryoserializer.buffer.max.mb", "2048")

In [ ]:
df = spark.read.parquet("s3a://d2k-datalake-raw-dev/on-prem-mssql-export/dena-bank/CustomerLoanDetailsForPD/*.parquet").limit(10)

In [ ]:
import mlflow
logged_model = '../dashboard/artifacts/LogisticRegression'

loaded_model = mlflow.spark.load_model(logged_model)

In [ ]:
loaded_model.tr

In [3]:
import pickle

# Path to the pickle file
file_path = "../dashboard/assets/encoding_list.pkl"

with open(file_path, "rb") as file:
    data = pickle.load(file)

In [ ]:
data[1]

In [5]:
categorical_cols = ['GL',
 'Area',
 'Caste',
 'Scheme',
 'Activity',
 'District',
 'Religion',
 'SubSector',
 'Occupation',
 'FORMATGROUP',
 'Constitution',
 'FACILITYTYPE',
 'CUSTOMERSTATUS',
 'FACILITIESSTATUS',
 ]


numerical_cols = ['Age',
 'CAD',
 'CADU',
 'UsedRv',
 'Balance',
 'Product',
 'TotOsNF',
 'AdhocAmt',
 'InttRate',
 'InttType',
 'TotalAdv',
 'AppGovGur',
 'CURQTRINT',
 'GLProduct',
 'PRVQTRINT',
 'RvPrimary',
 'Unsecured',
 'TotLimitNF',
 'CoverGovGur',
 'DepValPlant',
 'ACCTotalProv',
 'BaselCatMark',
 'CURRENTLIMIT',
 'CurQtrCredit',
 'DRAWINGPOWER',
 'PREQTRCREDIT',
 'UnAdjSubSidy',
 'UnappliedInt',
 'ProvUnsecured',
 'TotalWriteOff',
 'AppropriatedRv',
 'OrgCostOfEquip',
 'TotLimitFunded',
 'Cust_AssetClass',
 'LimitSanctioned',
 'PriCashSecurity',
 'CollCashSecurity',
 'CollNonCRMSecurity',
 'OrgCostOfPlantMech']


In [14]:
import pandas as pd
from collections import OrderedDict

columns = OrderedDict()

for col in categorical_cols:
    for data_item in data:
        if col in data_item:
            # Extract the values from the dictionary
            values = list(data_item[col].values())
            # Ensure all lists are of the same length by padding with None (or NaN)
            max_length = max(len(columns.get(k, [])) for k in columns) if columns else len(values)
            for k in columns:
                # Pad existing columns to match the new max length
                columns[k] += [None] * (max_length - len(columns[k]))
            # Add the new column
            columns[col] = values + [None] * (max_length - len(values))

# Handle any remaining columns that need padding
max_length = max(len(v) for v in columns.values())
for k in columns:
    columns[k] += [None] * (max_length - len(columns[k]))

# Create the DataFrame
df = pd.DataFrame(columns)


In [6]:
dropdown_options = [
    {col: list(data_item[col].values())}
    for col in categorical_cols
    for data_item in data
    if col in data_item
]

In [ ]:
dropdown_options[1]['Area'][0]

In [ ]:
len(data)

In [4]:
categorical_columns = {'GL': 'TL - - RETAIL SCHEMES ', 'Area': 'URBAN', 'Caste': 'OBC', 'Scheme': 'DENA GRADING & SORTING', 'Activity': 'PROCESSING OF PULSES E.G.  DAL MILLS', 'District': 'INDORE', 'Religion': 'HINDU', 'SubSector': 'SERVICES - MICRO ENTERPRISES-NPS', 'Occupation': 'SURVEYORS / ACTUARIES', 'FORMATGROUP': 'C', 'Constitution': 'CENTRAL GOVERNMENT', 'FACILITYTYPE': 'DL', 'CUSTOMERSTATUS': 'STD', 'FACILITIESSTATUS': 'STD'}

In [5]:
def find_encodings(categorical_columns, data):

    encoding = {}

    for col, value in categorical_columns.items():
        for index, encoding_dict in enumerate(data):
            if col in encoding_dict:
                for key, val in encoding_dict[col].items():
                    if val == value:
                        mean_encoding_key = f"{col}_mean_encoding"
                        encoding[col] = data[index][mean_encoding_key][key]
                                
    return encoding


In [6]:
encodings = find_encodings(categorical_columns, data)

In [ ]:
encodings

In [7]:
from pyspark.sql import Row
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession

In [8]:
def transform_numerical_columns(numerical_columns,df):

    # Assemble numerical columns into a single vector
    assembler = VectorAssembler(inputCols=list(numerical_columns.keys()), outputCol="features")
    assembled_df = assembler.transform(df)

    # Apply StandardScaler
    scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
    scaler_model = scaler.fit(assembled_df)
    scaled_df = scaler_model.transform(assembled_df)

    # Extract scaled values into a dictionary
    transformed_values = {}
    scaled_data = scaled_df.select("scaledFeatures").collect()
    for i, col in enumerate(numerical_columns.keys()):
        transformed_values[col] = scaled_data[0]["scaledFeatures"][i]

    return transformed_values

In [9]:

numerical_columns = {'Age': 30, 'CAD': 0, 'CADU': 0, 'UsedRv': 0, 'Balance': 0, 'Product': 0, 'TotOsNF': 0, 'AdhocAmt': 0, 'InttRate': 0, 'InttType': 0, 'TotalAdv': 0, 'AppGovGur': 0, 'CURQTRINT': 0, 'GLProduct': 0, 'PRVQTRINT': 0, 'RvPrimary': 0, 'Unsecured': 0, 'TotLimitNF': 0, 'CoverGovGur': 0, 'DepValPlant': 0, 'ACCTotalProv': 0, 'BaselCatMark': 0, 'CURRENTLIMIT': 0, 'CurQtrCredit': 0, 'DRAWINGPOWER': 0, 'PREQTRCREDIT': 0, 'UnAdjSubSidy': 0, 'UnappliedInt': 0, 'ProvUnsecured': 0, 'TotalWriteOff': 0, 'AppropriatedRv': 0, 'OrgCostOfEquip': 0, 'TotLimitFunded': 0, 'Cust_AssetClass': 0, 'LimitSanctioned': 0, 'PriCashSecurity': 0, 'CollCashSecurity': 0, 'CollNonCRMSecurity': 0, 'OrgCostOfPlantMech': 0}

df = spark.createDataFrame([numerical_columns])


In [ ]:
df.show()

In [10]:
transformed_values = transform_numerical_columns(numerical_columns, df)


In [ ]:
transformed_values

In [ ]:
final_data = {**encodings,**transformed_values}
final_data

In [ ]:
final_data_typed = {key: float(value) if isinstance(value, (int, float)) else value for key, value in final_data.items()}
final_data_typed

In [14]:

# Create a PySpark DataFrame
final_df = spark.createDataFrame([final_data_typed])
# final_df = spark.createDataFrame([final_data])

In [ ]:
final_df.show()

In [15]:
categorical_cols = ['GL',
 'Area',
 'Caste',
 'Scheme',
 'Activity',
 'District',
 'Religion',
 'SubSector',
 'Occupation',
 'FORMATGROUP',
 'Constitution',
 'FACILITYTYPE',
 'CUSTOMERSTATUS',
 'FACILITIESSTATUS',
 ]


numerical_cols = ['Age',
 'CAD',
 'CADU',
 'UsedRv',
 'Balance',
 'Product',
 'TotOsNF',
 'AdhocAmt',
 'InttRate',
 'InttType',
 'TotalAdv',
 'AppGovGur',
 'CURQTRINT',
 'GLProduct',
 'PRVQTRINT',
 'RvPrimary',
 'Unsecured',
 'TotLimitNF',
 'CoverGovGur',
 'DepValPlant',
 'ACCTotalProv',
 'BaselCatMark',
 'CURRENTLIMIT',
 'CurQtrCredit',
 'DRAWINGPOWER',
 'PREQTRCREDIT',
 'UnAdjSubSidy',
 'UnappliedInt',
 'ProvUnsecured',
 'TotalWriteOff',
 'AppropriatedRv',
 'OrgCostOfEquip',
 'TotLimitFunded',
 'Cust_AssetClass',
 'LimitSanctioned',
 'PriCashSecurity',
 'CollCashSecurity',
 'CollNonCRMSecurity',
 'OrgCostOfPlantMech']

In [ ]:
desired_order = numerical_cols + categorical_cols
desired_order

In [17]:
final_df = final_df.select([col for col in desired_order])

In [ ]:
final_df.show()

In [18]:
final_features = final_df.columns

In [ ]:
final_assembler = VectorAssembler(inputCols=final_features, outputCol="features")
what_if_data = final_assembler.transform(final_df)
pred = loaded_model.transform(what_if_data)
print(pred)

In [ ]:
from pyspark.sql.functions import col

In [31]:
probability_list = pred.select("probability").rdd.flatMap(lambda x: x).collect()

In [ ]:
round(probability_list[0][1],3)

In [ ]:
null_count = client.download_artifacts('2a1f6d3066984315af874674efc29f37',art[2].path,dst_path='../dashboard/artifacts',)

In [ ]:
import json

# Open and read the JSON file
with open('../dashboard/artifacts/first_five_rows.json', 'r') as file:
    data = json.load(file)

# Print the data
print(data)

In [106]:
df = pd.DataFrame(data["data"], columns=data["columns"])

In [ ]:
df

In [71]:
df = pd.DataFrame(list(data.items()), columns=["column", "null_count"])


In [62]:
import dash

In [ ]:
df

In [ ]:
data.values()

In [ ]:
for column, count in data.items():
    print(column,count)

In [ ]:
import yaml
with open('../config/config.yaml', 'r') as f:
    data = yaml.load(f, Loader=yaml.SafeLoader)
    
# Print the values as a dictionary
print(data)

In [ ]:
data['run_id']

### Backtesting

In [4]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("PySpark Test") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.memory", "10g") \
    .master("local[*]") \
    .getOrCreate()

In [ ]:
import mlflow
logged_model = '../dashboard/artifacts/LogisticRegression'

loaded_model = mlflow.spark.load_model(logged_model)

In [3]:
import pickle

# Path to the pickle file
file_path = "../dashboard/assets/encoding_list.pkl"

with open(file_path, "rb") as file:
    data = pickle.load(file)

In [4]:
categorical_cols = ['GL',
 'Area',
 'Caste',
 'Scheme',
 'Activity',
 'District',
 'Religion',
 'SubSector',
 'Occupation',
 'FORMATGROUP',
 'Constitution',
 'FACILITYTYPE',
 'CUSTOMERSTATUS',
 'FACILITIESSTATUS',
 ]


numerical_cols = ['Age',
 'CAD',
 'CADU',
 'UsedRv',
 'Balance',
 'Product',
 'TotOsNF',
 'AdhocAmt',
 'InttRate',
 'InttType',
 'TotalAdv',
 'AppGovGur',
 'CURQTRINT',
 'GLProduct',
 'PRVQTRINT',
 'RvPrimary',
 'Unsecured',
 'TotLimitNF',
 'CoverGovGur',
 'DepValPlant',
 'ACCTotalProv',
 'BaselCatMark',
 'CURRENTLIMIT',
 'CurQtrCredit',
 'DRAWINGPOWER',
 'PREQTRCREDIT',
 'UnAdjSubSidy',
 'UnappliedInt',
 'ProvUnsecured',
 'TotalWriteOff',
 'AppropriatedRv',
 'OrgCostOfEquip',
 'TotLimitFunded',
 'Cust_AssetClass',
 'LimitSanctioned',
 'PriCashSecurity',
 'CollCashSecurity',
 'CollNonCRMSecurity',
 'OrgCostOfPlantMech']


In [56]:
import pandas as pd
df = pd.read_csv('../dashboard/assets/200_sample.csv',encoding='utf-8')

In [ ]:
df.head()

In [51]:
def find_encodings(categorical_columns, data):

    encoding = {}

    for col, value in categorical_columns.items():
        for index, encoding_dict in enumerate(data):
            if col in encoding_dict:
                for key, val in encoding_dict[col].items():
                    if val == value:
                        mean_encoding_key = f"{col}_mean_encoding"
                        encoding[col] = data[index][mean_encoding_key][key]
                                
    return encoding


In [ ]:
len(df)

In [33]:
encoding_list = []
for i in range(len(df)):
    encoding = find_encodings(df.iloc[i], data)
    encoding_list.append(encodings)

In [36]:
encodings = pd.DataFrame(encoding_list)

In [ ]:
encodings

In [63]:
def transform_numerical_columns(numerical_columns,df):

    assembler = VectorAssembler(inputCols=list(numerical_columns.keys()), outputCol="features")
    assembled_df = assembler.transform(df)

    scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
    scaler_model = scaler.fit(assembled_df)
    scaled_df = scaler_model.transform(assembled_df)

    transformed_values = {}
    scaled_data = scaled_df.select("scaledFeatures").collect()
    for i, col in enumerate(numerical_columns.keys()):
        transformed_values[col] = scaled_data[0]["scaledFeatures"][i]

    return transformed_values

In [ ]:
numerical_columns = df[numerical_cols]
numerical_columns.head()

In [ ]:
numerical_columns = df[numerical_cols]
df = spark.createDataFrame(numerical_columns)


In [60]:
transformed_values = transform_numerical_columns(numerical_columns, df)


In [ ]:
transformed_values

In [43]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.sql import DataFrame

def apply_standard_scaler(df: DataFrame, input_cols: list, output_col: str) -> DataFrame:
    """
    Applies StandardScaler to the specified columns in a PySpark DataFrame.

    Parameters:
    - df (DataFrame): The input PySpark DataFrame.
    - input_cols (list): List of column names to scale.
    - output_col (str): The name of the output column containing the scaled values.

    Returns:
    - DataFrame: A new DataFrame with the scaled values in the specified output column.
    """
    from pyspark.sql import SparkSession
    from pyspark.ml.feature import StandardScaler, VectorAssembler
    
    # Combine input columns into a single vector column
    assembler = VectorAssembler(inputCols=input_cols, outputCol="features_vector")
    df = assembler.transform(df)
    
    # Initialize the StandardScaler
    scaler = StandardScaler(inputCol="features_vector", outputCol=output_col, withMean=True, withStd=True)
    
    # Fit and transform the DataFrame
    scaler_model = scaler.fit(df)
    scaled_df = scaler_model.transform(df)
    
    # Return the resulting DataFrame
    return scaled_df.select(*df.columns, output_col)

scaled_df = apply_standard_scaler(df, input_cols=numerical_cols, output_col="scaled_features")


In [ ]:
scaled_df.select('scaled_features').show()

In [46]:
encoding_df = spark.createDataFrame(encodings)

In [ ]:
encoding_df.show()

In [ ]:
import plotly.graph_objects as go

# Define the model's accuracy


# Show the plot
fig.show()


In [ ]:
import plotly.graph_objects as go

# Define the metrics and their values
metrics = ['F1 Score', 'FPR', 'TPR', 'Precision', 'ROC AUC']
values = [0.85, 0.12, 0.90, 0.88, 0.92]  # Replace with your values

# Ensure the radar plot is closed by repeating the first value
values += [values[0]]  # Close the loop for the radar chart
metrics += [metrics[0]]  # Close the loop for the radar chart

# Create the radar plot
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=values,
    theta=metrics,
    fill='toself',
    name='Model Metrics'
))

# Update the layout for better visualization
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]  # Assuming metric values range from 0 to 1
        )
    ),
    title="Radar Plot of Model Metrics",
    showlegend=False
)

# Show the plot
fig.show()


In [2]:
import json

path = '../dashboard/assets/roc_curve_XGBClassifier.json'
with open(path, 'r') as file:
        roc_data = json.load(file)

In [6]:
step = 300
fpr_reduced = roc_data["fpr"][::step]
tpr_reduced = roc_data["tpr"][::step]

In [ ]:
len(fpr_reduced)

In [ ]:
import plotly.graph_objs as go

path = '../dashboard/assets/roc_curve_LogisticRegression.json'
with open(path, 'r') as file:
    roc_data = json.load(file)
step = 300
fpr_reduced = roc_data["fpr"][::step]
tpr_reduced = roc_data["tpr"][::step]
# Plot the ROC curve
fig = go.Figure()

# Add the line for ROC
fig.add_trace(
    go.Scatter(
        x=fpr_reduced,
        y=tpr_reduced,
        mode="lines+markers",
        name="ROC Curve",
        line=dict(color="blue", width=2),
        marker=dict(size=6)
    )
)

# Add layout details
fig.update_layout(
    title="ROC Curve",
    xaxis_title="False Positive Rate (FPR)",
    yaxis_title="True Positive Rate (TPR)",
    xaxis=dict(range=[0, 1], showgrid=True),
    yaxis=dict(range=[0, 1], showgrid=True),
    template="plotly_white",
    showlegend=True
)

fig.show()



In [ ]:
path = '../dashboard/assets/feature_importances_LogisticRegression.json'
with open(path, 'r') as file:
    feature_importance_dict = json.load(file)

feature_importance_dict['coefficients']
# abs(feature_importance_dict['coefficients']['UsedRv'])
# sorted_features = sorted(feature_importance_dict['coefficients'].items(), key=lambda x: x[1], reverse=True)
# sorted_features

In [ ]:
path = '../dashboard/assets/feature_importances_LogisticRegression.json'
with open(path, 'r') as file:
    feature_importance_dict = json.load(file)
# Sort the dictionary by importance values in descending order
sorted_features = sorted(feature_importance_dict['coefficients'].items(), key=lambda x: x[1], reverse=True)

# Extract the top 10 features and their corresponding importance values
top_10_features = sorted_features[:10]
top_10_feature_names = [item[0] for item in top_10_features]
top_10_importance_values = [item[1] for item in top_10_features]
top_10_feature_names = top_10_feature_names[::-1]
top_10_importance_values = top_10_importance_values[::-1]

# Create the horizontal bar chart using Plotly
fig = go.Figure(go.Bar(
    x=top_10_importance_values,
    y=top_10_feature_names,
    orientation='h',  # Horizontal bars
    marker=dict(color='skyblue')
))

# Customize the layout
fig.update_layout(
    title="Top 10 Feature Importances",
    xaxis_title="Importance",
    yaxis_title="Features",
    template="plotly_dark",
    showlegend=False
)

fig.show()

In [1]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("PySpark Test") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.memory", "10g") \
    .master("local[*]") \
    .getOrCreate()

In [ ]:
import mlflow

model_path = '../dashboard/artifacts/XGBClassifier'
loaded_model = mlflow.spark.load_model(model_path)